In [1]:
%pip install contractions
%pip install datasets
%pip install Flask flask-ngrok flask-cors


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pyngrok


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

import re
import string # for text cleaning
import contractions # for expanding short form words
from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch

from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

/Users/shruti/Documents/MS_Hackathon/CV-JD-Matching/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!ngrok authtoken 2mPZwifpvClOzsoP4f51dmErsBm_6BguCKQ3qY3nztsE7Z1uJ

Authtoken saved to configuration file: /Users/shruti/Library/Application Support/ngrok/ngrok.yml


In [4]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('./serviceAccountKey.json')

firebase_admin.initialize_app(cred)

db = firestore.client()

# Get Resume

In [5]:
def get_resume_by_userId(userId):
    resumes_ref = db.collection('Resumes')
    query = resumes_ref.where('userId', '==', userId).get()
    if not query:
        print(f"No resume found for userId: {userId}")
        return None
    for doc in query:
        resume_data = doc.to_dict()
        return resume_data

user_id = "Shruti456"
resume = get_resume_by_userId(user_id)
if resume:
    print(f"Resume for {user_id}: {resume}")
else:
    print("No resume found.")

/Users/shruti/Documents/MS_Hackathon/CV-JD-Matching/venv/lib/python3.12/site-packages/google/cloud/firestore_v1/base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Resume for Shruti456: {'certifications': [{'title': 'Frontend Web Developer Nanodegree', 'date': '2022-08-01', 'issuer': 'Udacity'}, {'title': 'Responsive Web Design Certification', 'date': '2023-03-01', 'issuer': 'FreeCodeCamp'}], 'keywords': ['Frontend Development', 'JavaScript', 'React'], 'embeddingVector': [-0.07979843765497208, 0.08942137658596039, 0.07722501456737518, -0.06293588876724243, 0.30689430236816406, -0.2819562256336212, -0.19332920014858246, 0.3673371970653534, -0.11723736673593521, -0.11008210480213165, 0.03504513204097748, -0.3614444136619568, 0.018966952338814735, 0.34477725625038147, 0.055095549672842026, 0.17672695219516754, 0.23545274138450623, 0.12052682042121887, -0.18198886513710022, 0.19299104809761047, 0.010219422169029713, -0.10656534880399704, 0.02201114594936371, 0.491380512714386, 0.22704674303531647, 0.06906893104314804, 0.019056031480431557, 0.23572279512882233, -0.23872889578342438, -0.022725965827703476, 0.32009437680244446, 0.06870114058256149, -0.0

# Text Preprocessing
process_resume() and process_job_description(), both of which perform text cleaning on resumes and job descriptions. 
The text is cleaned by removing punctuation, URLs, emails, and numbers, while also expanding contractions and converting all text to lowercase. 
The cleaned text is stored back into the dictionaries under the key 'CV' for further use. This ensures that resumes and job descriptions are processed uniformly for comparison or analysis purposes.

In [8]:
import re
import string
import contractions

def text_cleaning(text: str) -> str:
    if not text:
        return ""
    text = text.lower().strip()
    translator = str.maketrans('', '', string.punctuation)
    text = contractions.fix(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text)
    text = text.translate(translator)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    return text.strip()

def process_resume(resume_dict):
    skills = ' '.join(resume_dict.get('skills', []))
    education_list = resume_dict.get('education', [])
    education_text = ' '.join([edu.get('degree', '') for edu in education_list])
    parsed_text = resume_dict.get('parsedText', '')
    combined_text = skills + ' ' + education_text + ' ' + parsed_text
    cleaned_text = text_cleaning(combined_text)
    resume_dict['CV'] = cleaned_text
    return resume_dict


def process_job_description(job_dict):
    job_title = job_dict.get('jobTitle', '')
    job_description = job_dict.get('jobDescription', '')
    responsibilities = ' '.join(job_dict.get('responsibilities', []))
    required_skills = ' '.join(job_dict.get('requiredSkills', []))
    combined_text = f"{job_title} {job_description} {responsibilities} {required_skills}"
    cleaned_text = text_cleaning(combined_text)
    job_dict['CV'] = cleaned_text
    return job_dict

processed_resume = process_resume(resume)
print(f"Cleaned CV for {processed_resume['userId']}: {processed_resume['CV']}")

Cleaned CV for Shruti456: javascript react css html bachelor of science in computer science shruti is a talented frontend engineer with a passion for creating beautiful user interfaces


 Using DistilBERT model from Hugging Face to generate embeddings for a given text. It tokenizes the input text, processes it through the model, and averages the last hidden states to produce a single embedding vector. The embedding can be used for tasks like text comparison or job-resume matching. The first 10 values of the embedding are printed for verification.

In [9]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

def generate_embedding(text: str):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertModel.from_pretrained('distilbert-base-uncased')
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    embedding = torch.mean(last_hidden_state, dim=1).squeeze()
    return embedding.tolist()

processed_resume_text = processed_resume['CV']
resume_embedding = generate_embedding(processed_resume_text)
print(f"Resume Embedding for {processed_resume['userId']}: {resume_embedding[:10]}...")

/Users/shruti/Documents/MS_Hackathon/CV-JD-Matching/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/shruti/Documents/MS_Hackathon/CV-JD-Matching/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Resume Embedding for Shruti456: [-0.07979832589626312, 0.08942128717899323, 0.07722491770982742, -0.06293599307537079, 0.30689433217048645, -0.2819562256336212, -0.19332917034626007, 0.36733731627464294, -0.11723732948303223, -0.11008191853761673]...


# Flask APIs


In [10]:
# Define the classes as dictionaries for simplicity
class Certification:
    def __init__(self, date, issuer, title):
        self.date = date
        self.issuer = issuer
        self.title = title

class Education:
    def __init__(self, degree, university, gradYear):
        self.degree = degree
        self.university = university
        self.gradYear = gradYear

class Experience:
    def __init__(self, company, role, duration):
        self.company = company
        self.role = role
        self.duration = duration

This Flask application provides various endpoints for job and resume management, including uploading resumes and job postings, matching jobs to candidates and vice versa. It utilizes Firebase Firestore as the database for storing jobs and resumes, employs cosine similarity to compare embedding vectors of resumes and job descriptions, and integrates Ngrok for exposing the service to the internet.







In [11]:
from flask import Flask, request, jsonify
import firebase_admin
from firebase_admin import credentials, firestore
from pyngrok import ngrok
from flask_cors import CORS, cross_origin
import uuid

# Initialize the Flask app and set CORS to allow cross-origin requests
portN = 7777
app = Flask(__name__)
CORS(app, support_credentials=True)

# Create a public URL using ngrok for exposing the Flask app
public_url = ngrok.connect(portN)
print(f"Public URL: {public_url}")

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec_a, vec_b):
    dot_product = np.dot(vec_a, vec_b)
    norm_a = np.linalg.norm(vec_a)
    norm_b = np.linalg.norm(vec_b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return dot_product / (norm_a * norm_b)

# Route for uploading a resume and processing it
@app.route('/upload_resume', methods=['POST'])
@cross_origin(supports_credentials=True)
def upload_resume():
    data = request.json
    required_fields = ['userId']

    # Ensure required fields are provided in the request
    for field in required_fields:
        if field not in data:
            return jsonify({'error': f'Missing field: {field}'}), 400

    try:
        # Extract and structure experience, education, and certifications data
        experience_list = [Experience(exp['company'], exp['role'], exp['duration']).__dict__ for exp in data['experience']]
        education_list = [Education(edu['degree'], edu['university'], edu['gradYear']).__dict__ for edu in data['education']]
        certification_list = [Certification(cert['date'], cert['issuer'], cert['title']).__dict__ for cert in data['certifications']]

        # Prepare resume data to store
        resume_data = {
            "resumeId": data['userId'],
            "userId": data['userId'],
            "contactInfo": data['contactInfo'],
            "preferredLocation": data['preferredLocation'],
            "languages": data['languages'],
            "skills": data['skills'],
            "experience": experience_list,
            "education": education_list,
            "certifications": certification_list,
            "fileURL": data['fileURL'],
            "parsedText": data['parsedText'],
            "keywords": data['keywords'],
        }

        # Process resume text, generate embedding, and store it in Firestore
        processed_resume = process_resume(resume_data)
        resume_embedding = generate_embedding(processed_resume['CV'])
        resume_data["embeddingVector"] = resume_embedding
        db.collection('Resumes').doc(resume_data['userId']).set(resume_data)
        return jsonify({'success': 'Resume uploaded successfully'}), 201
    except Exception as e:
        # Handle errors and return a message
        return jsonify({'error': str(e)}), 500

# Route for uploading a job post and processing it
@app.route('/upload_job', methods=['POST'])
@cross_origin(supports_credentials=True)
def upload_job():
    data = request.json
    required_fields = ['employerId']

    # Ensure required fields are provided in the request
    for field in required_fields:
        if field not in data:
            return jsonify({'error': f'Missing field: {field}'}), 400
    job_id = str(uuid.uuid4())

    try:
        # Prepare job data to store in Firestore
        job_data = {
            "jobId": job_id,
            "employerId": data['employerId'],
            "contactInfo": data['contactInfo'],
            "jobTitle": data['jobTitle'],
            "company": data['company'],
            "jobDescription": data['jobDescription'],
            "requiredSkills": data['requiredSkills'],
            "responsibilities": data['responsibilities'],
            "location": data['location'],
            "salaryRange": data['salaryRange'],
            "keywords": data['keywords'],
            "postDate": data['postDate'],
            "expirationDate": data['expirationDate'],
        }

        # Process job description, generate embedding, and store it
        processed_job = process_job_description(job_data)
        job_embedding = generate_embedding(processed_job['CV'])
        job_data["embeddingVector"] = job_embedding
        db.collection('Jobs').doc(job_id).set(job_data)
        return jsonify({'success': 'Job uploaded successfully'}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Route for finding top job matches based on a resume
@app.route('/find_top_jobs', methods=['POST'])
@cross_origin(supports_credentials=True)
def find_top_jobs():
    data = request.json

    if 'userId' not in data:
        return jsonify({'error': 'Missing userId'}), 400
    user_id = data['userId']

    # Fetch the user's resume from Firestore
    resume_ref = db.collection('Resumes').document(user_id)
    resume_doc = resume_ref.get()

    if resume_doc.exists:
        resume_data = resume_doc.to_dict()
        if 'embeddingVector' not in resume_data or not resume_data['embeddingVector']:
            # Generate embedding if not available
            processed_resume = process_resume(resume_data)
            resume_embedding = generate_embedding(processed_resume['CV'])
            resume_ref.update({'embeddingVector': resume_embedding})
        else:
            resume_embedding = resume_data['embeddingVector']
    else:
        return jsonify({'error': 'Resume not found for userId: {}'.format(user_id)}), 404

    # Fetch all jobs and calculate cosine similarity with the resume
    job_collection = db.collection('Jobs').stream()
    similarities = []

    for job in job_collection:
        job_data = job.to_dict()
        job_embedding = job_data.get('embeddingVector')

        if not job_embedding:
            processed_job = process_job_description(job_data)
            job_embedding = generate_embedding(processed_job['CV'])
            db.collection('Jobs').document(str(job_data['jobId'])).update({'embeddingVector': job_embedding})

        score = cosine_similarity(resume_embedding, job_embedding)
        similarities.append((job_data['jobId'], score))

    # Sort by similarity score and retrieve top jobs
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_jobs = [job_id for job_id, score in similarities[:5]]

    resume_ref.update({'matches': top_jobs})

    jobs_list = []
    for job_id in top_jobs:
        job_ref = db.collection('Jobs').document(job_id)
        job_doc = job_ref.get()

        if job_doc.exists:
            job_data = job_doc.to_dict()
            job_data.pop('embeddingVector', None)
            jobs_list.append(job_data)
        else:
            return jsonify({'error': 'Job not found for jobId: {}'.format(job_id)}), 404

    return jsonify({'jobs': jobs_list}), 200

# Route for finding top candidates based on a job
@app.route('/find_top_candidates', methods=['POST'])
@cross_origin(supports_credentials=True)
def find_top_candidates():
    data = request.json

    if 'jobId' not in data:
        return jsonify({'error': 'Missing jobId'}), 400
    job_id = data['jobId']

    # Fetch the job document from Firestore
    job_ref = db.collection('Jobs').document(job_id)
    job_doc = job_ref.get()

    if job_doc.exists:
        job_data = job_doc.to_dict()

        job_embedding = job_data.get('embeddingVector')
        if not job_embedding:
            processed_job = process_job_description(job_data)
            job_embedding = generate_embedding(processed_job['CV'])
            job_ref.update({'embeddingVector': job_embedding})
    else:
        return jsonify({'error': 'Job not found for jobId: {}'.format(job_id)}), 404

    # Fetch resumes and calculate cosine similarity with the job
    resume_collection = db.collection('Resumes').stream()
    similarities = []

    for resume in resume_collection:
        resume_data = resume.to_dict()
        resume_embedding = resume_data.get('embeddingVector')

        if not resume_embedding:
            processed_resume = process_resume(resume_data)
            resume_embedding = generate_embedding(processed_resume['CV'])
            db.collection('Resumes').document(resume.id).update({'embeddingVector': resume_embedding})

        score = cosine_similarity(job_embedding, resume_embedding)
        similarities.append((resume.id, score))

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_candidates = [candidate_id for candidate_id, score in similarities[:5]]

    candidates_list = []
    for candidate_id in top_candidates:
        # Retrieve resume for each candidate
        resume_ref = db.collection('Resumes').document(candidate_id)
        resume_doc = resume_ref.get()

        if resume_doc.exists:
            candidate_data = resume_doc.to_dict()
            candidate_data.pop('embeddingVector', None)
            candidates_list.append(candidate_data)
        else:
            return jsonify({'error': 'Candidate not found for candidateId: {}'.format(candidate_id)}), 404

    return jsonify({'candidates': candidates_list}), 200

# Basic hello world route
@app.route('/hello', methods=['GET'])
@cross_origin(supports_credentials=True)
def hello_world():
    return "Hello, World!"

# Start the Flask app on port 7777
app.run(host='0.0.0.0', port=portN)


Public URL: NgrokTunnel: "https://a7da-2601-c0-ca7f-ec10-682f-f8f0-94b-2cd6.ngrok-free.app" -> "http://localhost:7777"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7777
 * Running on http://10.0.0.252:7777
Press CTRL+C to quit


In [25]:
ngrok.kill()